Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import torch
import rlcard
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [ ]:
# Make environment
env = rlcard.make('kuhn-poker', config={'seed': 0})
eval_env = rlcard.make('kuhn-poker', config={'seed': 0})

In [ ]:
# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 100
evaluate_num = 1000
episode_num = 10000

In [ ]:
# The paths for saving the logs and learning curves
log_dir = './experiments/kuhn_poker_dqn_result/'

In [ ]:
# Set a global seed
set_global_seed(0)

In [ ]:
agent = DQNAgent(env.action_num, env.state_shape[0])
random_agent = RandomAgent(action_num=eval_env.action_num)
env.set_agents([agent, agent])
eval_env.set_agents([agent, random_agent])

In [ ]:
# Init a Logger to plot the learning curve
logger = Logger(log_dir)

In [ ]:
for episode in range(episode_num):
    # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

    # Feed transitions into agent memory, and train the agent
    for ts in trajectories[0]:
        agent.train(ts)

    # Evaluate the performance. Play with random agents.
    if episode % evaluate_every == 0:
        logger.log_performance(episode,
                               tournament(eval_env, evaluate_num)[0])

In [ ]:
# Close files in the logger
logger.close_files()

In [ ]:
# Plot the learning curve
logger.plot('DQN')

In [ ]:
# Save model
save_dir = 'models/kuhn_poker_dqn'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
state_dict = agent.get_state_dict()

In [ ]:
torch.save(state_dict, os.path.join(save_dir, 'model.pth'))

In [ ]:
agent.weight_updates